In [ ]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime 

import model_based_agent as mba 
import worm_env as we 
import ensemble_mod_env as eme

from improc import *
import utils as ut
import tab_agents as tab

In [ ]:
folder = './Data/Rand'+datetime.now().strftime('%d-%m-%H-%M')+'/'
if os.path.isdir(folder):
    raise ValueError('Folder already exists.')
os.mkdir(folder)

worm_num = 0
worm = we.ProcessedWorm(0,ep_len=300)

In [ ]:
fname = f'{folder}Rand{worm_num}.pkl'
mba.get_init_traj(fname, worm, 6)
worm_num += 1